In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1317
┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1317


In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0
n = 500

data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [5]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  12.28 MiB
  allocs estimate:  206592
  --------------
  minimum time:     13.965 ms (0.00% GC)
  median time:      15.845 ms (0.00% GC)
  mean time:        17.660 ms (12.34% GC)
  maximum time:     27.742 ms (34.23% GC)
  --------------
  samples:          283
  evals/sample:     1

In [6]:
inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=1.008122852504758, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=2.0081228525047585, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=3.008122852504758, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=4.008122852504757, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.0081228525047585, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=6.008122852504759, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=7.008122852504758, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=8.008122852504757, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=9.008122852504753, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.008122852504755, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=11.008122852504755, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float6